In [7]:
import pathlib
import typing

import dask
import dask.array
import dask.dataframe
import dask.bag
import polyflexmd.data_analysis.data.read as read
import polyflexmd.data_analysis.data.constants as constants
import polyflexmd.data_analysis.transform.transform as transform
import polyflexmd.data_analysis.pipelines.trajectory

import pandas as pd

import re

import MDAnalysis as mda

In [57]:
import dask.utils
dask.bag.read_text(
    "/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=1/j_d_end=1/polymer-1-1.out",
    collection=False
)

[Delayed('list-1044d4bae61963cf2c95e58538f18008')]

In [2]:
%load_ext autoreload
%autoreload 2

In [14]:
paths_trajs = [
    pathlib.Path(
        "/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=2/j_d_end=1/polymer-2-1.out"),
    pathlib.Path(
        "/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=2/j_d_end=2/polymer-2-2.out")
]
paths_trajs

[PosixPath('/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=2/j_d_end=1/polymer-2-1.out'),
 PosixPath('/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=2/j_d_end=2/polymer-2-2.out')]

In [48]:
topology = mda.topology.LAMMPSParser.DATAParser(
    "/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/initial_system.data"
).parse()
topology.bonds

ValueError: invalid literal for int() with base 10: '508      bonds\n'

In [5]:
system = read.read_lammps_system_data(pathlib.Path(
    "/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/initial_system.data"))

In [17]:
system.atoms

,molecule-ID,type,x,y,z,ix,iy,iz
1,1,1,0.000000,0.000000,0.000000,0,0,0
2,1,1,-0.159927,-0.017451,-0.956566,0,0,0
3,1,2,-0.086992,-0.765936,-0.343908,0,0,0
4,1,2,0.059319,0.029401,-0.879569,0,0,0
5,1,2,0.514538,0.823298,-0.558005,0,0,0
...,...,...,...,...,...,...,...,...
508,4,2,11.500683,-0.771811,-0.504867,0,0,0
509,4,2,10.579229,-0.598039,-0.753109,0,0,0
510,4,2,10.236980,-1.059966,0.028165,0,0,0
511,4,2,10.520269,-1.676332,0.721519,0,0,0


In [13]:
df_bag = dask.bag.read_text(paths_trajs, linedelimiter="\n", blocksize="128MiB", include_path=True).to_dataframe(
    columns=["row", "path"]).head(100)
df_bag

,row,path
0,ITEM: TIMESTEP\n,/home/egor/Projects/polyflexmd/data/test-5-FEN...
1,1000000\n,/home/egor/Projects/polyflexmd/data/test-5-FEN...
2,ITEM: NUMBER OF ATOMS\n,/home/egor/Projects/polyflexmd/data/test-5-FEN...
3,512\n,/home/egor/Projects/polyflexmd/data/test-5-FEN...
4,ITEM: BOX BOUNDS pp pp pp\n,/home/egor/Projects/polyflexmd/data/test-5-FEN...
...,...,...
95,87 2 17.669 -2.46987 -6.22782 0 0 0\n,/home/egor/Projects/polyflexmd/data/test-5-FEN...
96,88 2 16.9546 -1.81579 -6.48717 0 0 0\n,/home/egor/Projects/polyflexmd/data/test-5-FEN...
97,89 2 16.343 -1.71131 -7.26439 0 0 0\n,/home/egor/Projects/polyflexmd/data/test-5-FEN...
98,90 2 16.3116 -1.4149 -8.14843 0 0 0\n,/home/egor/Projects/polyflexmd/data/test-5-FEN...


In [24]:
def process_timestep(df: dask.dataframe.DataFrame, path_to_var: dict[str, read.VariableTrajectoryPath]):

    timestep = df.iloc[1][0]
    columns = df.iloc[8][0].split()[2:]

    path = df.iloc[0]["path"]
    variable_names, variable_values = zip(*path_to_var[path].variables)

    header = ["t", *columns, *variable_names]
    rows = []

    for _, row in df.iloc[9:].iterrows():
        values = row["row"].split()
        values.insert(0, timestep)
        values.extend(variable_values)
        rows.append(values)

    return pd.DataFrame(rows, columns=header).astype(constants.RAW_TRAJECTORY_DF_COLUMN_TYPES)


def read_lammps_trajectories(
        paths: list[read.VariableTrajectoryPath],
        column_types: dict = constants.RAW_TRAJECTORY_DF_COLUMN_TYPES,
        time_steps_per_partition: int = 100000,
) -> dask.dataframe.DataFrame:
    print(f"Reading {paths}...")

    paths_d = {str(vtp_path): vtp for vtp in paths for vtp_path in vtp.paths}
    variable_names = (name for name, _ in paths[0].variables)
    paths = [p for vtp in paths for p in vtp.paths]

    df_bag = dask.bag.read_text(paths, linedelimiter="\n", blocksize="128MiB", include_path=True).to_dataframe(columns=["row", "path"])

    index = df_bag["row"].str.contains("ITEM: TIMESTEP").cumsum()
    df_bag["i"] = index
    df_bag = df_bag.set_index("i", sorted=True, sort=False).repartition(divisions=index.unique().compute().sort_values().tolist())
    df_bag.persist()

    columns = df_bag.loc[df_bag["row"].str.contains("ITEM: ATOMS")].head(1).iloc[0]["row"].split()[2:]
    columns.insert(0, "t")

    df: dask.dataframe.DataFrame = df_bag.groupby(["i", "path"]).apply(
        process_timestep,
        path_to_var=paths_d,
        meta=pd.DataFrame(columns=[*columns, *variable_names]).astype(column_types)
    )
    #print(f"Indexing dataframe from {path}...")
    divisions = df["t"].loc[df["t"] % time_steps_per_partition == 0].unique().compute().sort_values().tolist()
    df = df.set_index("t", divisions=divisions)

    return df

vtps = [
    read.VariableTrajectoryPath(
        variables=[("kappa", 1.0), ("d_end", 1.2)],
        possible_values=[],
        paths=[pathlib.Path("/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=2/j_d_end=1/polymer-2-1.out")]
    ),
    read.VariableTrajectoryPath(
        variables=[("kappa", 1.0), ("d_end", 1.4)],
        possible_values=[],
        paths=[pathlib.Path("/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=2/j_d_end=2/polymer-2-2.out")]
    )
]
read_lammps_trajectory(vtps).compute()

Reading [VariableTrajectoryPath(variables=[('kappa', 1.0), ('d_end', 1.2)], possible_values=[], paths=[PosixPath('/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=2/j_d_end=1/polymer-2-1.out')]), VariableTrajectoryPath(variables=[('kappa', 1.0), ('d_end', 1.4)], possible_values=[], paths=[PosixPath('/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=2/j_d_end=2/polymer-2-2.out')])]...


,id,type,x,y,z,ix,iy,iz,kappa,d_end
t,,,,,,,,,,
1000000,1,1,0.000000,0.00000,0.00000,0,0,0,1.0,1.4
1000000,162,2,-13.374000,7.13641,-10.26180,0,0,0,1.0,1.2
1000000,163,2,-12.423400,7.13054,-9.91717,0,0,0,1.0,1.2
1000000,164,2,-11.614300,6.97447,-9.50037,0,0,0,1.0,1.2
1000000,165,2,-10.982700,6.46110,-9.02570,0,0,0,1.0,1.2
...,...,...,...,...,...,...,...,...,...,...
2000000,4,2,-0.397928,1.57091,-1.87166,0,0,0,1.0,1.2
2000000,5,2,-0.499559,2.47456,-2.05904,0,0,0,1.0,1.2
2000000,6,2,-0.495117,3.11698,-1.33814,0,0,0,1.0,1.2


In [11]:
next(read_lammps_custom_trajectory_file_generator(path=pathlib.Path(p),
                                                  column_types=constants.RAW_TRAJECTORY_DF_COLUMN_TYPES
                                                  ))

NotImplementedError: Iteration of DataFrameGroupBy objects requires computing the groups which may be slow. You probably want to use 'apply' to execute a function for all the columns. To access individual groups, use 'get_group'. To list all the group names, use 'df[<group column>].unique().compute()'.

t      NaN
id     NaN
type   NaN
x      NaN
y      NaN
z      NaN
ix     NaN
iy     NaN
iz     NaN
dtype: float64

In [8]:
kappas = [1.0 + i * 5 for i in range(2)]
d_ends = [1.2 + i * 0.2 for i in range(3)]

df_trajectories = polyflexmd.data_analysis.pipelines.trajectory.read_and_process_trajectories(
    trajectories=polyflexmd.data_analysis.data.read.get_experiment_trajectories_paths(
        experiment_raw_data_path=pathlib.Path(
            "/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw"),
        style="l_K+d_end",
        kappas=kappas,
        d_ends=d_ends,
        read_relax=True
    ),
    system=system
)

df_trajectories

Reading paths [PosixPath('/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=1/j_d_end=1/polymer_relax-1-1.out'), PosixPath('/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=1/j_d_end=1/polymer-1-1.out')] ...
Joining ...
Unfolding coordinates...
Reading paths [PosixPath('/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=1/j_d_end=2/polymer_relax-1-2.out'), PosixPath('/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=1/j_d_end=2/polymer-1-2.out')] ...
Joining ...
Unfolding coordinates...
Reading paths [PosixPath('/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=1/j_d_end=3/polymer_relax-1-3.out'), PosixPath('/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=1/j_

,t,id,type,x,y,z,molecule-ID,kappa,d_end
npartitions=672,,,,,,,,,
,uint64,uint16,uint8,float64,float64,float64,uint16,category[known],category[known]
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [14]:
df_trajectories.to_csv(
    "/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/processed/trajectories.csv",
    single_file=True, index=False)

['/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/processed/trajectories.csv']

In [12]:
df_ete = transform.calc_end_to_end_df(
    df_trajectories,
    group_by_params=["kappa", "d_end"],
    parallel=False
)
df_ete

ValueError: Metadata inference failed in `groupby.apply(lambda)`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
IndexError('single positional indexer is out-of-bounds')

Traceback:
---------
  File "/home/egor/Projects/polyflexmd/.venv/lib/python3.10/site-packages/dask/dataframe/utils.py", line 193, in raise_on_meta_error
    yield
  File "/home/egor/Projects/polyflexmd/.venv/lib/python3.10/site-packages/dask/dataframe/groupby.py", line 2489, in apply
    meta = self._meta_nonempty.apply(func, *meta_args, **meta_kwargs)
  File "/home/egor/Projects/polyflexmd/.venv/lib/python3.10/site-packages/pandas/core/groupby/groupby.py", line 1353, in apply
    result = self._python_apply_general(f, self._selected_obj)
  File "/home/egor/Projects/polyflexmd/

In [10]:
df_trajectory_unfolded = transform.unfold_coordinates_df(
    trajectory_df=transform.join_raw_trajectory_df_with_system_data(
        raw_trajectory_df=read.read_multiple_raw_trajectory_dfs([
            pathlib.Path(
                "/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=1/j_d_end=1/polymer_relax-1-1.out"),
            pathlib.Path(
                "/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=1/j_d_end=1/polymer-1-1.out")
        ]),
        system_data=system
    ),
    system_data=system
)
df_trajectory_unfolded

Reading paths [PosixPath('/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=1/j_d_end=1/polymer_relax-1-1.out'), PosixPath('/home/egor/Projects/polyflexmd/data/test-5-FENE-beadspring-vary-l_K-vary-d_end/e296c212/data/raw/i_kappa=1/j_d_end=1/polymer-1-1.out')] ...
Joining ...
Unfolding coordinates...


,t,id,type,x,y,z,ix,iy,iz,molecule-ID
npartitions=19152,,,,,,,,,,
,uint64,uint16,uint8,float64,float64,float64,int16,int16,int16,uint16
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
